## Step 1 Fetch Stock Prices
We will use the NVIDIA and CocaCola stock historical data for the analysis, as well as getting the historical data of the NASDAQ market. The stock history period ranges from 2023-01-01 to 2024-01-01

In [1]:
import yfinance as yf

In [2]:
# Define the stock symbol and the time period
nvidia_stock_symbol = "NVDA"  # NVIDIA Ltd.
coca_stock_symbol = "KO"  # Coca Cola Ltd.
market_symbol = "^NDX"  # NASDAQ 100

start_date = "2023-01-01"
end_date = "2024-01-01"

In [3]:
# Fetch historical stock data
nvidia_stock_data = yf.download(nvidia_stock_symbol, start=start_date, end=end_date)
coca_stock_data = yf.download(coca_stock_symbol, start=start_date, end=end_date)
market_data = yf.download(market_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
nvidia_stock_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03,148.509995,149.960007,140.960007,143.149994,143.079773,40127700
2023-01-04,145.669998,148.529999,142.410004,147.490005,147.417679,43132400
2023-01-05,144.910004,145.639999,141.479996,142.649994,142.580048,38916800
2023-01-06,144.740005,150.100006,140.339996,148.589996,148.517120,40504400
2023-01-09,152.839996,160.559998,151.410004,156.279999,156.203354,50423100
...,...,...,...,...,...,...
2023-12-22,491.950012,493.829987,484.670013,488.299988,488.277069,25213900
2023-12-26,489.679993,496.000000,489.600006,492.790009,492.766907,24420000
2023-12-27,495.109985,496.799988,490.850006,494.170013,494.146820,23364800


In [5]:
# Extract the closing prices
nvidia_adj_close = nvidia_stock_data["Adj Close"].pct_change().dropna()
coca_adj_close = coca_stock_data["Adj Close"].pct_change().dropna()
market_adj_close = market_data["Adj Close"].pct_change().dropna()

# verify the integrity of the closing price data
print(nvidia_adj_close.shape)
print(coca_adj_close.shape)
print(market_adj_close.shape)

(249,)
(249,)
(249,)


## Step 2 Calculate the Beta Coefficient
Recall that the beta coefficient $\beta=\dfrac{Covariance(R_e,R_m)}{Variance(R_m)}$, where $R_e$ is the return on an individual stock and $R_m$ is the return of the overall market.

In [6]:
import numpy as np

In [7]:
covariance_nvidia = np.cov(nvidia_adj_close, market_adj_close)[0, 1]
covariance_coca = np.cov(coca_adj_close, market_adj_close)[0, 1]
variance_market = np.var(market_adj_close)

In [8]:
beta_nvidia = covariance_nvidia / variance_market
beta_coca = covariance_coca / variance_market

In [9]:
print(f"The beta of {nvidia_stock_symbol} in relation to {market_symbol} is: {beta_nvidia:.4f}")
print(f"The beta of {coca_stock_symbol} in relation to {market_symbol} is: {beta_coca:.4f}")

The beta of NVDA in relation to ^NDX is: 1.8392
The beta of KO in relation to ^NDX is: 0.1676
